In [1]:

from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import numpy as np
import zipfile

import csv
import urllib3

In [2]:

url = 'http://www.fueleconomy.gov/feg/download.shtml'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')


reg = 'epadata/\d{2}data.zip' ##luckily our target has a unique identifier for href
suf = 'http://www.fueleconomy.gov/feg/'

In [4]:
links = soup.find_all('a')
zlist = []
for tag in links:
    link = tag.get('href',None)
    if link is not None:
        if re.match(reg, link):
            zlist.append(link)

In [18]:

for x in zlist:
    url = suf+x
    localfn = url.split('/')[-1]
    yearno = int(localfn[:2])
    if yearno < 50 and yearno > 00:
        r = requests.get(url, stream=True)
        with open(localfn, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
        zip_ref = zipfile.ZipFile(localfn, 'r')
        zip_ref.extractall("dldir")
        zip_ref.close()


NotImplementedError: compression type 6 (implode)